Comando abaixo apenas para que seja possível importar os modulos corretamente

In [1]:
import sys
sys.path.append("../../") 

# Criação de Features de Estilo

Nessa seção explicaremos como implementar as diferentes features de estilo.

## Como criar uma ParagraphBasedFeature

Uma ParagraphBasedFeature é uma classe de features textuais que calcula o resultado das métricas utilizando os parágrafos do texto como parâmetro. Para criar uma feature paragraphbased, fazemos

###### Importamos os módulos necessários:

In [2]:
from feature.features import ParagraphBasedFeature

###### Criamos a classe responsável pela feature:

In [3]:
class EnemQuality(ParagraphBasedFeature):
    
    '''informa a qualidade de uma redação do Enem pelo quesito Coesão, em porcentagem'''
    
    def __init__(self,name,description,reference,visibility,text_format,feature_time_per_document):
        super(ParagraphBasedFeature,self).__init__(name,description,reference,visibility,text_format,feature_time_per_document)
        self.int_percent = 0
        self.int_parag_counter = 0

A classe de exemplo acima herda de ParagraphBasedFeature. Seus atributo são:

* int_percent: responsável por armazenar o percentual de qualidade do texto;
* int_parag_counter: conta o número de parágrafos do texto.

###### Implementamos os métodos abstratos

Observemos a classe CharBasedFeature. A classe contém métodos abstratos que devem ser implementados nas classes filhas.

In [4]:
from feature.features import FeatureCalculator
from abc import abstractmethod

class ParagraphBasedFeature(FeatureCalculator):

    @abstractmethod
    def checkParagraph(self,document,paragraph):
        raise NotImplementedError

Como observado, a classe ParagraphBasedFeature herda de FeatureCalculator. A classe FeatureCalculator também possui métodos abstratos que devem ser implementados. Abaixo o diagrama de classes que representa a estrutura de hierarquia.

![title](pbf.jpeg)

Em resumo, os métodos a serem implementados são checkParagraph, compute_feature e finish_document. O método checkParagraph é responsável por verificar cada parágrafo do texto. O método compute_feature é responsável por calcular o resultado da feature desejada, e o finish_document responsável por resetar as variáveis utilizadas no cálculo da feature.

Vamos implementar esses métodos na subclasse ParagraphBasedFeature de exemplo.

In [5]:
class EnemQuality(ParagraphBasedFeature):
    
    '''informa a qualidade de uma redação do Enem pelo quesito Coesão, em porcentagem'''
    
    def __init__(self,name,description,reference,visibility,text_format,feature_time_per_document):
        super(ParagraphBasedFeature,self).__init__(name,description,reference,visibility,text_format,feature_time_per_document)
        self.int_percent = 0
        self.int_parag_counter = 0
    
    def checkParagraph(self,document,paragraph):
        self.int_parag_counter = self.int_parag_counter + 1
    
    def compute_feature(self,document):
        if self.int_parag_counter <= 1:
            self.int_percent = 0
        elif self.int_parag_counter is 2:
            self.int_percent = 15
        elif self.int_parag_counter is 3:
            self.int_percent = 60
        elif self.int_parag_counter is 4:
            self.int_percent = 100
        elif self.int_parag_counter is 5:
            self.int_percent = 50
        else:
            self.int_percent = 0
        
        return self.int_percent

    def finish_document(self,document):
        self.int_parag_counter = 0
        self.int_percent = 0
        

Seguindo esses passos, é possível implementar uma feature ParagraphBasedFeature.

### Testando a feature implementada

Para fazer o teste manual da classe implementada, executamos seus métodos nas suas respectivas funções. Abaixo um código de teste para a execução da feature, em teste unitário.

In [6]:
import unittest
from feature.features import FeatureVisibilityEnum, Document, FeatureCalculator
from utils.basic_entities import FormatEnum, FeatureTimePerDocumentEnum

class TestFeatureCalculator(unittest.TestCase):
    
    def testParagraphCountTest(self):

            pcount = EnemQuality("EnemQuality", "Informa a qualidade de uma redação do Enem pelo quesito Coesão, em porcentagem", 
                                             "Jupyter Notebook documentation, text in https://g1.globo.com/educacao/noticia/2019/10/08/mec-lanca-programa-para-estimular-o-ensino-tecnico.ghtml", 
                                             FeatureVisibilityEnum.public, 
                                             FormatEnum.text_plain, 
                                             FeatureTimePerDocumentEnum.MILLISECONDS)

            document = Document(1,"doc1","O texto nao precisa -necessariamente - ser o texto que sera testado")
            
            par = str("Um dos trabalhos usados pelo governo federal para embasar o programa Novos Caminhos,"
            + " apresentado no site do MEC, apresenta dados que mostram que " +
            '"a realização de cursos de formação profissional não proporciona, necessariamente, '
            + 'maior facilidade para a inserção ocupacional".')
            
            pcount.checkParagraph(document, par)
            
            par = str("A conclusão da nota técnica 199, de novembro e 1998,"
            + "do Departamento Intersindical de Estatísticas e Estudos Socioeconômicos (Dieese)"
            + " é que isso se deve à característica do mercado brasileiro, mais voltada ao setor de serviços,"
            + " que não exigem formação técnica.")
            
            pcount.checkParagraph(document, par)
            
            par = str('"Entre as 20 ocupações mais comuns entre os que concluíram o Técnico,'
            + " treze são também as mais recorrentes entre o total de ocupados com nível médio. "
            + "Essa constatação leva à reflexão sobre a configuração do mercado de trabalho brasileiro,"
            + " que se baseia principalmente em postos de trabalho dos setores do comércio e de serviços, "
            + "que não exigem esse tipo de qualificação, possuem longas jornadas, baixa remuneração e"
            + ' recebem a maior parte dos jovens no primeiro emprego", afirma o texto.')
            
            pcount.checkParagraph(document, par)
            
            int_result = pcount.compute_feature(document)
            print("A qualidade do texto é de " + str(int_result) + "%.")
            pcount.finish_document(document)
            self.assertEqual(int_result, 60, "A porcentagem de qualidade do primeiro documento está incorreta.")
    
sys.argv = ['', 'TestFeatureCalculator.testParagraphCountTest']
unittest.main()

.

A qualidade do texto é de 60%.



----------------------------------------------------------------------
Ran 1 test in 0.001s

OK


SystemExit: False

/usr/lib/python3/dist-packages/IPython/core/interactiveshell.py:2889: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
O resultado do compute_feature deve ser 60, assim como o teste unitário.